In [50]:
import pandas as pd
from qdrant_client import QdrantClient
from sentence_transformers import SentenceTransformer
import numpy as np
from tqdm import tqdm
from qdrant_client.models import Distance, VectorParams, PointStruct, TextIndexParams, TokenizerType
import os
import torch
import uuid


In [51]:
print(torch.cuda.is_available())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

True


device(type='cuda')

In [53]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
model = model.to(device)

client = QdrantClient(url="http://localhost:6333")

if (client.collection_exists(collection_name="amazon_products")):
    client.delete_collection(collection_name="amazon_products")
    
collection_name = "amazon_products"

In [54]:

df_products = pd.read_csv(r'C:\Users\dczqd\Documents\test_product_reco\data\archive\amazon_products.csv')
df_category = pd.read_csv(r'C:\Users\dczqd\Documents\test_product_reco\data\archive\amazon_categories.csv')
df_combined = pd.merge(df_products, df_category, left_on='category_id', right_on='id')
df_combined = df_combined.drop(columns=['id', 'category_id', 'productURL']).dropna()
df_combined

,asin,title,imgUrl,stars,reviews,price,listPrice,isBestSeller,boughtInLastMonth,category_name
0,B014TMV5YE,"Sion Softside Expandable Roller Luggage, Black...",https://m.media-amazon.com/images/I/815dLQKYIY...,4.5,0,139.99,0.00,False,2000,Suitcases
1,B07GDLCQXV,Luggage Sets Expandable PC+ABS Durable Suitcas...,https://m.media-amazon.com/images/I/81bQlm7vf6...,4.5,0,169.99,209.99,False,1000,Suitcases
2,B07XSCCZYG,Platinum Elite Softside Expandable Checked Lug...,https://m.media-amazon.com/images/I/71EA35zvJB...,4.6,0,365.49,429.99,False,300,Suitcases
3,B08MVFKGJM,Freeform Hardside Expandable with Double Spinn...,https://m.media-amazon.com/images/I/91k6NYLQyI...,4.6,0,291.59,354.37,False,400,Suitcases
4,B01DJLKZBA,Winfield 2 Hardside Expandable Luggage with Sp...,https://m.media-amazon.com/images/I/61NJoaZcP9...,4.5,0,174.99,309.99,False,400,Suitcases
...,...,...,...,...,...,...,...,...,...,...
1426332,B00R3LIKCO,American Flag Patriotic USA Classic 5 Panel Me...,https://m.media-amazon.com/images/I/71PDJFz6AA...,4.2,0,14.95,0.00,False,0,Men's Accessories
1426333,B098BQ7ZQ3,Men's Baseball Cap - H2O-DRI Line Up Curved Br...,https://m.media-amazon.com/images/I/812Tycexs4...,4.4,0,33.99,0.00,False,0,Men's Accessories
1426334,B07X1MVNT1,[4 Pack] Adjustable Eyeglasses and Sunglasses ...,https://m.media-amazon.com/images/I/61vvYW1S9J...,3.6,0,8.54,0.00,False,0,Men's Accessories
1426335,B08XLBG8V9,Ax2002 Aviator Sunglasses,https://m.media-amazon.com/images/I/51+yjD4F1x...,4.5,0,54.36,57.39,False,0,Men's Accessories


In [55]:
electronics = ['Car Electronics & Accessories',
  'Computer Servers',
  'Computer Monitors',
  'Computers & Tablets',
  'Computer Networking',
  'Computer Components',
  'Computer External Components',
  'Headphones & Earbuds',
  'Office Electronics',
  'Portable Audio & Video',
  'Cell Phones & Accessories',
  'Vehicle Electronics',
  'Camera & Photo',
  'Computers',
  'Home Audio & Theater Products',
  'Smart Home: Security Cameras and Systems',
  "Kids' Electronics"]
active_wear= ['Motorcycle & Powersports',
  'Sports Nutrition Products',
  'Sports & Fitness',
  'Sports & Outdoors',
  'Sports & Outdoor Play Toys',
  'Wearable Technology']
bags =  ['Travel Duffel Bags',
  'Messenger Bags',
  'Suitcases',
  'Travel Tote Bags',
  'Garment Bags',
  'Luggage Sets',
  'Backpacks',
  'Luggage',
  'Laptop Bags',
  "Women's Handbags"]
df_combined['classified_category'] = df_combined['category_name'].apply(lambda x: 'electronics' if x in electronics else ('active_wear' if x in active_wear else ('bags' if x in bags else 'others')))
df_combined_small = df_combined[df_combined['classified_category'] != 'others']
# df_combined_small =     df_combined.sample(n=200000, random_state=42)

df_combined_small['combined_features'] = df_combined_small.apply(lambda x: f"{x['title']} {x['category_name']}".lower(), axis=1)

df_combined_small


C:\Users\dczqd\AppData\Local\Temp\ipykernel_32752\1186570285.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_combined_small['combined_features'] = df_combined_small.apply(lambda x: f"{x['title']} {x['category_name']}".lower(), axis=1)


,asin,title,imgUrl,stars,reviews,price,listPrice,isBestSeller,boughtInLastMonth,category_name,classified_category,combined_features
0,B014TMV5YE,"Sion Softside Expandable Roller Luggage, Black...",https://m.media-amazon.com/images/I/815dLQKYIY...,4.5,0,139.99,0.00,False,2000,Suitcases,bags,"sion softside expandable roller luggage, black..."
1,B07GDLCQXV,Luggage Sets Expandable PC+ABS Durable Suitcas...,https://m.media-amazon.com/images/I/81bQlm7vf6...,4.5,0,169.99,209.99,False,1000,Suitcases,bags,luggage sets expandable pc+abs durable suitcas...
2,B07XSCCZYG,Platinum Elite Softside Expandable Checked Lug...,https://m.media-amazon.com/images/I/71EA35zvJB...,4.6,0,365.49,429.99,False,300,Suitcases,bags,platinum elite softside expandable checked lug...
3,B08MVFKGJM,Freeform Hardside Expandable with Double Spinn...,https://m.media-amazon.com/images/I/91k6NYLQyI...,4.6,0,291.59,354.37,False,400,Suitcases,bags,freeform hardside expandable with double spinn...
4,B01DJLKZBA,Winfield 2 Hardside Expandable Luggage with Sp...,https://m.media-amazon.com/images/I/61NJoaZcP9...,4.5,0,174.99,309.99,False,400,Suitcases,bags,winfield 2 hardside expandable luggage with sp...
...,...,...,...,...,...,...,...,...,...,...,...,...
1419457,B0BV6SR3DH,Camocon Retro Flood Small Pritnted Icon Tote B...,https://m.media-amazon.com/images/I/71Vlxh4Qqd...,0.0,0,59.00,0.00,False,0,Women's Handbags,bags,camocon retro flood small pritnted icon tote b...
1419458,B078HLV6XG,Women's Hand Painted Genuine Leather Wristlet ...,https://m.media-amazon.com/images/I/A1gNIKHUKV...,4.9,0,53.17,60.80,False,0,Women's Handbags,bags,women's hand painted genuine leather wristlet ...
1419459,B0BZ3LCH8T,MRS Purse for Bride Acrylic Clutch Bridal Show...,https://m.media-amazon.com/images/I/71wQIZv6BW...,3.5,0,15.99,16.99,False,0,Women's Handbags,bags,mrs purse for bride acrylic clutch bridal show...
1419460,B09PZJ17QD,Ruby Top Zip Hobo,https://m.media-amazon.com/images/I/710h0EGfpS...,5.0,0,69.00,0.00,False,0,Women's Handbags,bags,ruby top zip hobo women's handbags


In [56]:
df_combined_small.to_csv('df_combined_small.csv', index=False)
df_combined_small


,asin,title,imgUrl,stars,reviews,price,listPrice,isBestSeller,boughtInLastMonth,category_name,classified_category,combined_features
0,B014TMV5YE,"Sion Softside Expandable Roller Luggage, Black...",https://m.media-amazon.com/images/I/815dLQKYIY...,4.5,0,139.99,0.00,False,2000,Suitcases,bags,"sion softside expandable roller luggage, black..."
1,B07GDLCQXV,Luggage Sets Expandable PC+ABS Durable Suitcas...,https://m.media-amazon.com/images/I/81bQlm7vf6...,4.5,0,169.99,209.99,False,1000,Suitcases,bags,luggage sets expandable pc+abs durable suitcas...
2,B07XSCCZYG,Platinum Elite Softside Expandable Checked Lug...,https://m.media-amazon.com/images/I/71EA35zvJB...,4.6,0,365.49,429.99,False,300,Suitcases,bags,platinum elite softside expandable checked lug...
3,B08MVFKGJM,Freeform Hardside Expandable with Double Spinn...,https://m.media-amazon.com/images/I/91k6NYLQyI...,4.6,0,291.59,354.37,False,400,Suitcases,bags,freeform hardside expandable with double spinn...
4,B01DJLKZBA,Winfield 2 Hardside Expandable Luggage with Sp...,https://m.media-amazon.com/images/I/61NJoaZcP9...,4.5,0,174.99,309.99,False,400,Suitcases,bags,winfield 2 hardside expandable luggage with sp...
...,...,...,...,...,...,...,...,...,...,...,...,...
1419457,B0BV6SR3DH,Camocon Retro Flood Small Pritnted Icon Tote B...,https://m.media-amazon.com/images/I/71Vlxh4Qqd...,0.0,0,59.00,0.00,False,0,Women's Handbags,bags,camocon retro flood small pritnted icon tote b...
1419458,B078HLV6XG,Women's Hand Painted Genuine Leather Wristlet ...,https://m.media-amazon.com/images/I/A1gNIKHUKV...,4.9,0,53.17,60.80,False,0,Women's Handbags,bags,women's hand painted genuine leather wristlet ...
1419459,B0BZ3LCH8T,MRS Purse for Bride Acrylic Clutch Bridal Show...,https://m.media-amazon.com/images/I/71wQIZv6BW...,3.5,0,15.99,16.99,False,0,Women's Handbags,bags,mrs purse for bride acrylic clutch bridal show...
1419460,B09PZJ17QD,Ruby Top Zip Hobo,https://m.media-amazon.com/images/I/710h0EGfpS...,5.0,0,69.00,0.00,False,0,Women's Handbags,bags,ruby top zip hobo women's handbags


In [57]:
batch_size = 512
embeddings = []
def encode_batch(model, batch, device):
    return model.encode(batch, device=device)

for i in tqdm(range(0, len(df_combined_small), batch_size), desc="Encoding rows"):
    batch = df_combined_small['combined_features'].iloc[i:i + batch_size].tolist()
    batch_embeddings = encode_batch(model, batch, device)
    embeddings.extend(batch_embeddings)

# Convert the list of embeddings to a DataFrame
df_combined_small['embeddings'] = embeddings



Encoding rows: 100%|██████████| 347/347 [00:57<00:00,  5.99it/s]
C:\Users\dczqd\AppData\Local\Temp\ipykernel_32752\1987695823.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_combined_small['embeddings'] = embeddings


In [58]:
df_combined_small.head()

,asin,title,imgUrl,stars,reviews,price,listPrice,isBestSeller,boughtInLastMonth,category_name,classified_category,combined_features,embeddings
0,B014TMV5YE,"Sion Softside Expandable Roller Luggage, Black...",https://m.media-amazon.com/images/I/815dLQKYIY...,4.5,0,139.99,0.00,False,2000,Suitcases,bags,"sion softside expandable roller luggage, black...","[-0.033734273, 0.025797043, -0.08109724, 0.044..."
1,B07GDLCQXV,Luggage Sets Expandable PC+ABS Durable Suitcas...,https://m.media-amazon.com/images/I/81bQlm7vf6...,4.5,0,169.99,209.99,False,1000,Suitcases,bags,luggage sets expandable pc+abs durable suitcas...,"[-0.05983128, 0.003999212, -0.068765745, 0.046..."
2,B07XSCCZYG,Platinum Elite Softside Expandable Checked Lug...,https://m.media-amazon.com/images/I/71EA35zvJB...,4.6,0,365.49,429.99,False,300,Suitcases,bags,platinum elite softside expandable checked lug...,"[-0.02091951, -0.0009106894, -0.049572278, 0.0..."
3,B08MVFKGJM,Freeform Hardside Expandable with Double Spinn...,https://m.media-amazon.com/images/I/91k6NYLQyI...,4.6,0,291.59,354.37,False,400,Suitcases,bags,freeform hardside expandable with double spinn...,"[-0.045961194, -0.074926734, -0.023375783, 0.0..."
4,B01DJLKZBA,Winfield 2 Hardside Expandable Luggage with Sp...,https://m.media-amazon.com/images/I/61NJoaZcP9...,4.5,0,174.99,309.99,False,400,Suitcases,bags,winfield 2 hardside expandable luggage with sp...,"[-0.03667262, 0.0098515265, -0.034812428, 0.03..."


In [60]:
import random

def hash_to_uuid(unique_string):
    return str(uuid.uuid5(uuid.NAMESPACE_DNS, unique_string))
def create_point(row):
  
    return PointStruct(
        id=hash_to_uuid(row['asin']),
        vector=np.array(row['embeddings']).tolist(), 
        payload={
            'asin' : row['asin'],
            'title' : row['title'],
            'imgUrl' : row['imgUrl'], 
            'stars' : float(row['stars']),  
            'reviews' : int(row['reviews']),
            'price' : float(row['price']), 
            'category_name': str(row['classified_category']),
            'isBestSeller' : bool(row['isBestSeller']),
            'boughtInLastMonth' : int(row['boughtInLastMonth']),
            'origin_country' :  random.choice(['Singapore', 'China'])

         
        }
    )

points = []
for index, row in tqdm(df_combined_small.iterrows()):   
    point = create_point(row)
    points.append(point)
# try
# tqdm.pandas(desc="Processing rows")
# points = df_combined_small.progress_apply(create_point_from_row, axis=1).tolist()
print(len(points))


177442it [00:15, 11431.55it/s]

177442


In [61]:
response = client.create_collection(
    collection_name=f"{collection_name}",
    vectors_config=VectorParams(size=len(df_combined_small['embeddings'].tolist()[0]), distance=Distance.COSINE),
)
print(response)
client.get_collection(collection_name)

True


CollectionInfo(status=<CollectionStatus.GREEN: 'green'>, optimizer_status=<OptimizersStatusOneOf.OK: 'ok'>, vectors_count=None, indexed_vectors_count=0, points_count=0, segments_count=8, config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=384, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None, datatype=None, multivector_config=None), shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=None), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0), quant

In [62]:
print('dimensions',len(df_combined_small['embeddings'].tolist()[0]))

dimensions 384


In [63]:
df_combined_small

,asin,title,imgUrl,stars,reviews,price,listPrice,isBestSeller,boughtInLastMonth,category_name,classified_category,combined_features,embeddings
0,B014TMV5YE,"Sion Softside Expandable Roller Luggage, Black...",https://m.media-amazon.com/images/I/815dLQKYIY...,4.5,0,139.99,0.00,False,2000,Suitcases,bags,"sion softside expandable roller luggage, black...","[-0.033734273, 0.025797043, -0.08109724, 0.044..."
1,B07GDLCQXV,Luggage Sets Expandable PC+ABS Durable Suitcas...,https://m.media-amazon.com/images/I/81bQlm7vf6...,4.5,0,169.99,209.99,False,1000,Suitcases,bags,luggage sets expandable pc+abs durable suitcas...,"[-0.05983128, 0.003999212, -0.068765745, 0.046..."
2,B07XSCCZYG,Platinum Elite Softside Expandable Checked Lug...,https://m.media-amazon.com/images/I/71EA35zvJB...,4.6,0,365.49,429.99,False,300,Suitcases,bags,platinum elite softside expandable checked lug...,"[-0.02091951, -0.0009106894, -0.049572278, 0.0..."
3,B08MVFKGJM,Freeform Hardside Expandable with Double Spinn...,https://m.media-amazon.com/images/I/91k6NYLQyI...,4.6,0,291.59,354.37,False,400,Suitcases,bags,freeform hardside expandable with double spinn...,"[-0.045961194, -0.074926734, -0.023375783, 0.0..."
4,B01DJLKZBA,Winfield 2 Hardside Expandable Luggage with Sp...,https://m.media-amazon.com/images/I/61NJoaZcP9...,4.5,0,174.99,309.99,False,400,Suitcases,bags,winfield 2 hardside expandable luggage with sp...,"[-0.03667262, 0.0098515265, -0.034812428, 0.03..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1419457,B0BV6SR3DH,Camocon Retro Flood Small Pritnted Icon Tote B...,https://m.media-amazon.com/images/I/71Vlxh4Qqd...,0.0,0,59.00,0.00,False,0,Women's Handbags,bags,camocon retro flood small pritnted icon tote b...,"[-0.071907036, 0.041109696, -0.033777278, 0.07..."
1419458,B078HLV6XG,Women's Hand Painted Genuine Leather Wristlet ...,https://m.media-amazon.com/images/I/A1gNIKHUKV...,4.9,0,53.17,60.80,False,0,Women's Handbags,bags,women's hand painted genuine leather wristlet ...,"[-0.057686087, 0.020773256, 0.011585774, 0.023..."
1419459,B0BZ3LCH8T,MRS Purse for Bride Acrylic Clutch Bridal Show...,https://m.media-amazon.com/images/I/71wQIZv6BW...,3.5,0,15.99,16.99,False,0,Women's Handbags,bags,mrs purse for bride acrylic clutch bridal show...,"[-0.054684155, 0.07710012, 0.019617254, 0.0628..."
1419460,B09PZJ17QD,Ruby Top Zip Hobo,https://m.media-amazon.com/images/I/710h0EGfpS...,5.0,0,69.00,0.00,False,0,Women's Handbags,bags,ruby top zip hobo women's handbags,"[-0.07016895, 0.057026226, -0.069443226, 0.051..."


In [64]:
print(points[0])

id='7fcf8c73-c453-5c1e-8fe5-9f7cfa7b3206' vector=[-0.0337342731654644, 0.0257970429956913, -0.08109723776578903, 0.04435734823346138, 0.06161680445075035, -0.03724166750907898, 0.05457957834005356, 0.10229621082544327, -0.03188153728842735, -0.013579988852143288, 0.0604657344520092, 0.04835694283246994, 0.0013983536045998335, -0.0011845845729112625, -0.003874959656968713, 0.06157148256897926, 0.02204369753599167, 0.020132219418883324, -0.11646188050508499, 0.02987092174589634, -0.05153266713023186, -0.07031653076410294, -0.03801903501152992, 0.002507225377485156, -0.0791022926568985, 0.019551314413547516, 0.01415116898715496, -0.015076498501002789, 0.04057767242193222, -0.027917928993701935, 0.0013767718337476254, -0.027120210230350494, -0.021601127460598946, 0.10562282055616379, 0.15445907413959503, -0.00701109180226922, -0.005974262487143278, -0.022979453206062317, -0.06030365452170372, -0.05907105281949043, 0.0200759619474411, -0.00843257736414671, 0.004375705029815435, 0.0618863552

In [65]:

def upload_batches(points, batch_size=100):

    total_batches = len(points) // batch_size + (1 if len(points) % batch_size > 0 else 0)

    for i in tqdm(range(total_batches), desc="Uploading batches"):
        start_index = i * batch_size
        end_index = start_index + batch_size
        batch_points = points[start_index:end_index]
 
        response = client.upsert(
            collection_name=f"{collection_name}",
            points=batch_points
        )

    

In [66]:
upload_batches(points, batch_size=400)


Uploading batches: 100%|██████████| 444/444 [02:22<00:00,  3.12it/s]


In [70]:

client.create_payload_index(
    collection_name=f"{collection_name}",
    field_name="title",
    field_schema=TextIndexParams(
        type="text",
        tokenizer=TokenizerType.WORD,
        min_token_len=1,
        max_token_len=20,
        lowercase=True,
    ),
)


UpdateResult(operation_id=451, status=<UpdateStatus.COMPLETED: 'completed'>)

In [ ]:
# client.create_payload_index(
#     collection_name=f"{collection_name}",
#     field_name="category_name",
#     field_schema=TextIndexParams(
#         type="text",
#         tokenizer=TokenizerType.WORD,
#         min_token_len=1,
#         max_token_len=10,
#         lowercase=True,
#     ),
# )

UpdateResult(operation_id=525, status=<UpdateStatus.COMPLETED: 'completed'>)